In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("input2 (1).csv")

In [7]:
df = df.fillna("")

In [8]:
df

,id,name,time,pre
0,0,A,8,
1,1,B,2,
2,2,C,5,0
3,3,D,4,1
4,4,E,6,2 3
5,5,F,7,4
6,6,G,7,5


In [9]:
df["pre"] = df["pre"].transform(lambda x : (x.split()))
df["pre"] = df["pre"].transform(lambda x : np.array(x, dtype=int))

In [10]:
df

,id,name,time,pre
0,0,A,8,[]
1,1,B,2,[]
2,2,C,5,[0]
3,3,D,4,[1]
4,4,E,6,"[2, 3]"
5,5,F,7,[4]
6,6,G,7,[5]


In [11]:
df["suc"]  = df["pre"]

In [12]:
for i in range(len(df)):
    df.at[i, "suc"] = []

In [13]:
suc = df["suc"]

for i in range(len(df)):
    for x in df.at[i, "pre"]:
        suc[x].append(i)
df["suc"] = suc


In [14]:
df

,id,name,time,pre,suc
0,0,A,8,[],[2]
1,1,B,2,[],[3]
2,2,C,5,[0],[4]
3,3,D,4,[1],[4]
4,4,E,6,"[2, 3]",[5]
5,5,F,7,[4],[6]
6,6,G,7,[5],[]


In [15]:
df["start_forward"] = np.NAN
df["end_forward"] = np.NaN
df["start_backward"] = np.NaN
df["end_backward"] = np.NaN
df["slack_time"] = np.NAN

In [16]:
for i in range(len(df)):
    st = 0
    for x in df.at[i, "pre"]:
        st = max(st, df.at[x, "end_forward"])
    
    df.at[i, "start_forward"] = st
    df.at[i, "end_forward"] = st + df.at[i, "time"]


In [17]:
df

,id,name,time,pre,suc,start_forward,end_forward,start_backward,end_backward,slack_time
0,0,A,8,[],[2],0.0,8.0,NaN,NaN,NaN
1,1,B,2,[],[3],0.0,2.0,NaN,NaN,NaN
2,2,C,5,[0],[4],8.0,13.0,NaN,NaN,NaN
3,3,D,4,[1],[4],2.0,6.0,NaN,NaN,NaN
4,4,E,6,"[2, 3]",[5],13.0,19.0,NaN,NaN,NaN
5,5,F,7,[4],[6],19.0,26.0,NaN,NaN,NaN
6,6,G,7,[5],[],26.0,33.0,NaN,NaN,NaN


In [18]:
end_time = max(df["end_forward"])
print(end_time)

33.0


In [19]:
for i in range(len(df)-1 , -1, -1):
    et = end_time
    # print(df.at[i, "suc"])
    for x in df.at[i, "suc"]:
        et = min(et, df.at[x , "start_backward"])
    
    df.at[i, "end_backward"] = et
    df.at[i, "start_backward"] = et - df.at[i, "time"]
    df.at[i, "slack_time"] = et - df.at[i, "end_forward"]
    

In [20]:
df

,id,name,time,pre,suc,start_forward,end_forward,start_backward,end_backward,slack_time
0,0,A,8,[],[2],0.0,8.0,0.0,8.0,0.0
1,1,B,2,[],[3],0.0,2.0,7.0,9.0,7.0
2,2,C,5,[0],[4],8.0,13.0,8.0,13.0,0.0
3,3,D,4,[1],[4],2.0,6.0,9.0,13.0,7.0
4,4,E,6,"[2, 3]",[5],13.0,19.0,13.0,19.0,0.0
5,5,F,7,[4],[6],19.0,26.0,19.0,26.0,0.0
6,6,G,7,[5],[],26.0,33.0,26.0,33.0,0.0


In [21]:
cpm = []
for i in range(len(df)):
    if(df.at[i, "slack_time"] <= 0):
        cpm.append(df.at[i, "name"])


In [22]:
df

,id,name,time,pre,suc,start_forward,end_forward,start_backward,end_backward,slack_time
0,0,A,8,[],[2],0.0,8.0,0.0,8.0,0.0
1,1,B,2,[],[3],0.0,2.0,7.0,9.0,7.0
2,2,C,5,[0],[4],8.0,13.0,8.0,13.0,0.0
3,3,D,4,[1],[4],2.0,6.0,9.0,13.0,7.0
4,4,E,6,"[2, 3]",[5],13.0,19.0,13.0,19.0,0.0
5,5,F,7,[4],[6],19.0,26.0,19.0,26.0,0.0
6,6,G,7,[5],[],26.0,33.0,26.0,33.0,0.0


In [23]:
print("CPM is : " + " -> ".join(cpm))

CPM is : A -> C -> E -> F -> G
